# Data Handling with Pandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Handle import errors

Use the excel file '1-Data-errors.xlsx' to explore common data/datetime handling errors.

Practical = knowing when to python and when not to python.

In [ ]:
error_df = pd.read_excel(
    r'1-Data-errors.xlsx', 
    sheet_name='Sheet1',
    #header=1, # removes row 0, makes row 1 the headder
    #na_values=['na', '', ' '] # replaces value with NaN
    )

In [ ]:
error_df

In [ ]:
error_df.info()

In [ ]:
# Find the offending value

test_col = error_df['Data_with_errors'].values

for value in test_col:
    try:
        float(value)
    except:
        print(value)

# Go back to excel and CTRL + F for the offending value

In [ ]:
print(error_df['Data_clean']) # This works

In [ ]:
#print(error_df('Header_error')) # But this does not... why?

In [ ]:
# import same data as csv and look at the datetime


In [ ]:
df_errors_from_csv = pd.read_csv(r'1-Data-errors.csv')

In [ ]:
df_errors_from_csv

In [ ]:
df_errors_from_csv.info()

Useful blog post https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e

In [ ]:
df_errors_from_csv = pd.read_csv(
    r'1-Data-errors.csv',
    parse_dates = ['Date_best','Date_international','Date_USA'] # default is to parse as US format
    )

In [ ]:
df_errors_from_csv

In [ ]:
df_errors_from_csv = pd.read_csv(
    r'1-Data-errors.csv',
    parse_dates = ['Date_best','Date_international','Date_USA'],
    dayfirst=True # switch to parse as international format
    )

In [ ]:
df_errors_from_csv

Note how in all cases the "Date_best" format worked. This format saves you and your international collaborators / clients a lot of pain. 

## Import case study data

In [ ]:
well_data = pd.read_excel(r'1-Data-case-study.xlsx', sheet_name='ops_data')

In [ ]:
well_data.info()

## Quick-plot data using Pandas

**Syntax of plot()**

DataFrame.plot(x=None, y=None, **kwargs)

In [ ]:
well_data.plot(
    #x='Date',
    )

In [ ]:
#well_data.plot.scatter(x='Date_daily')

well_data.plot(
    x='Date',
    marker='.',
    markersize=2,
    linewidth=0.1,
    )

In [ ]:
well_data.columns

In [ ]:
well_data.plot(
    x='Date',
    y=['I-2 TMF T/hr'],
    marker='.',
    markersize=2,
    linewidth=0.1,
    )

# Add grid, google kwargs

## Calculate values

Calculations can be done row-wise using the column names

In [ ]:
data = [10,20,30]
df = pd.DataFrame(data, columns=['x'])
df

In [ ]:
df['x + 10'] = df['x'] + 10
df

When doing calculations there must be values in the cells (i.e., no NaN)

In [ ]:
well_data['Total_prod_tph'] = well_data['P-1 TMF T/hr'].fillna(0) + well_data['P-2 TMF T/hr'].fillna(0) + well_data['P-3 TMF T/hr'].fillna(0)

well_data['Total_inj_tph'] = well_data['I-1 TMF T/hr'].fillna(0) + well_data['I-2 TMF T/hr'].fillna(0)

In [ ]:
well_data.plot(
    x='Date', 
    y=['Total_prod_tph','Total_inj_tph'],
    linewidth=0.5
    )


## Averaging by time period

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

- D : daily
- W : weekly
- M : month end 
- SM : semi-month end (15th and end of month)
- Q : quarter end

In [ ]:

# Reassign to new object and set date the index
well_data_weekly_ave = well_data.set_index('Date')

# Resample (average) on weekly frequency
well_data_weekly_ave = well_data_weekly_ave.resample('W').mean() 

# Return the date index to be a column
well_data_weekly_ave.reset_index(inplace=True) # "inplace=True" instead of re-assignment 

display(well_data_weekly_ave.tail(3))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,4))

ax.scatter(
    well_data['Date'].values,
    well_data['Total_inj_tph'].values,
    s = 15,
    label = 'Daily total injection'
)

ax.scatter(
    well_data_weekly_ave['Date'].values,
    well_data_weekly_ave['Total_inj_tph'].values,
    s = 15,
    label = 'Weekly total injection'
)

ax.legend()
ax.set_ylabel('TMF [T/hr]')


## Filter data

Make a new dataframe that meets a criteria

Select subset of columns:
    
    new_df = old_df[list of new cols] 

In [ ]:
total_production = well_data_weekly_ave[['Date', 'Total_prod_tph']]

total_production.info()

In [ ]:
total_production['Exp_inj_TPH'] = total_production.Total_prod_tph * 0.85

Select all columns based on a criteria:

    new_df = old_df[boolean statement]

In [ ]:
low_total_production = total_production[total_production['Total_prod_tph'] < 300] # whats missing from here?

In [ ]:
low_total_production['Exp_inj_TPH'] = low_total_production.Total_prod_tph * 0.85

## Export results

In [ ]:
well_data_weekly_ave.to_csv('1-Data-weekly.csv', index=False)
#well_data_weekly_ave.to_excel('1-Data-weekly.xlsx', index=False)